In [4]:
import pandas as pd
import numpy as np
from pid_sm import PIDController
from concurrent.futures import ProcessPoolExecutor

# Function to simulate the system response using the PID controller
def simulate_system_response(pid_controller, setpoint, object_centers):
    data = {'Time': [], 'Input': [], 'Output': []}

    for object_center in object_centers:
        error = setpoint - object_center
        output = pid_controller.update(error)

        # Simulate time increments of 0.5 seconds
        time_increment = 0.5
        data['Time'].append(len(data['Time']) * time_increment)
        data['Input'].append(error)
        data['Output'].append(output)

    return pd.DataFrame(data)

def evaluate_parameters(kp, ki, kd, setpoint, object_centers):
    pid_controller = PIDController(kP=kp, kI=ki, kD=kd)
    pid_controller.reset()
    df = simulate_system_response(pid_controller, setpoint, object_centers)
    
    # Check if the output remains within the range [-90, 90]
    if np.max(df['Output']) <= 110 and np.min(df['Output']) >= -110:
        # Return a DataFrame with separate rows for each input and output value
        return pd.DataFrame({'kP': [kp] * len(df), 'kI': [ki] * len(df), 'kD': [kd] * len(df),
                             'Input': df['Input'].values, 'Output': df['Output'].values})
    else:
        # Return None when output is outside the desired range
        return None

# Generate input values
object_centers = np.arange(0, 641, 10)

# Setpoint for all cases
setpoint = 320

# Define the ranges for tuning parameters
kp_values = np.linspace(0.1, 2.0, 20)
ki_values = np.linspace(0.01, 0.5, 10)
kd_values = np.linspace(0.0, 0.5, 10)

# List to store DataFrames for valid tuning parameter sets
valid_params_dfs = []

# Using ProcessPoolExecutor for parallelization
with ProcessPoolExecutor() as executor:
    # Iterate through possible combinations of kp, ki, and kd
    futures = []
    for kp in kp_values:
        for ki in ki_values:
            for kd in kd_values:
                futures.append(executor.submit(evaluate_parameters, kp, ki, kd, setpoint, object_centers))

    # Retrieve results from futures
    for future in futures:
        params_df = future.result()

        # Check if the result is not None (output within the range [-90, 90])
        if params_df is not None:
            valid_params_dfs.append(params_df)

# Concatenate the list of valid parameter DataFrames into a single DataFrame
valid_params_df = pd.concat(valid_params_dfs, ignore_index=True)

# Save the DataFrame to an Excel file
excel_filename = 'valid_tuning_params.xlsx'
valid_params_df.to_excel(excel_filename, index=False)

# Print the head of the DataFrame
print("\nHead of the DataFrame:")
print(valid_params_df.head())

print(f'Data saved to {excel_filename}')



Head of the DataFrame:
    kP    kI   kD  Input  Output
0  0.1  0.01  0.0    320   33.60
1  0.1  0.01  0.0    310   34.15
2  0.1  0.01  0.0    300   34.65
3  0.1  0.01  0.0    290   35.10
4  0.1  0.01  0.0    280   35.50
Data saved to valid_tuning_params.xlsx
